# Recap

In order of priority/time taken

1. pandas init dict
    - `basal_area_aw_df = pd.DataFrame(columns=['BA_Aw'], index=xrange(max_age))`
    - find a faster way to create this data frame
    - relax the tolerance for aspen
2. pandas set item
    - use at method 
    - http://pandas.pydata.org/pandas-docs/stable/indexing.html#fast-scalar-value-getting-and-setting
3. lambdas
    - use cython for the gross tot vol and merch vol functions
    - might be wise to refactor these first to have conventional names, keyword arguments, and a base implementation to get rid of the boilerplate
    - don't be deceived - the callable is a miniscule portion; series.__getitem__ is taking most of the time
    - again, using .at here would probably be a significant improvement
4. basalareaincremementnonspatialaw
    - this is actually slow because of the number of times the BAFromZeroToDataAw function is called as shown above
    - relaxing the tolerance may help
    - indeed the tolerance is 0.01 * some value while the other factor finder functions have 0.1 tolerance i think
    - can also use cython for the increment functions

do a profiling run with IO (of reading input data and writing the plot curves to files) in next run


# Characterize what is happening

Indexing with df[] or series[] is slow for scalars (lambdas, pandas set)
basalareaincrement is running a lot for aw, use the same tolerance as is used for other species

merchvol, increment, and gross vol functions use pure python. cython would be effective.

# Decide on the action

- use same tolerance for aw as other species
- use at instead of [] or ix? - compare these in MWE
- creating data frame is slow, maybe because its fromdict. see if this can be improved

# MWEs

In [10]:
import pandas as pd
import numpy as np

## init from dict and xrange index vs from somethign else

### Timings

In [8]:
%%timeit
d = pd.DataFrame(columns=['A'], index=xrange(1000))

The slowest run took 38.51 times longer than the fastest. This could mean that an intermediate result is being cached.
1000 loops, best of 3: 368 µs per loop


In [9]:
%%timeit
d = pd.DataFrame(columns=['A'], index=xrange(1000), dtype='float')

The slowest run took 4.03 times longer than the fastest. This could mean that an intermediate result is being cached.
1000 loops, best of 3: 418 µs per loop


In [13]:
%%timeit
d = pd.DataFrame({'A': np.zeros(1000)})

The slowest run took 5.08 times longer than the fastest. This could mean that an intermediate result is being cached.
1000 loops, best of 3: 275 µs per loop


The problem here is that dataframe init being called 7000 times because of the aw ba factor finder

Maybe it's not worth using a data frame here. use a list or numpy and then convert to dataframe when the factor is found, e.g.:

In [14]:
%%timeit
for _ in xrange(5000):
    d = pd.DataFrame(columns=['A'], index=xrange(1000))

1 loop, best of 3: 2.13 s per loop


In [17]:
%%timeit
for _ in xrange(5000):
    d = np.zeros(1000)

100 loops, best of 3: 14.1 ms per loop


### Review the code to see how this can be applied

The numpy/purepython approach as potential

But there's a couple issues for which the code must be examined

The problem comes from the following call chain

`simulate_forwards_df` (called 1x) ->  
`get_factors_for_all_species` (called 10x, 1x per plot) ->  
`BAfactorFinder_Aw` (called 2x, 1x per plot that has aw) ->  
`BAfromZeroToDataAw` (called 7191 times, most of which in this chain) ->   
`DataFrame.__init__` (called 7932 times, most of which in this chain) ...  

why does `BAfromZeroToDataAw` create a dataframe? It's good to see the code:

First, `simulate_forwards_df` calls `get_factors_for_all_species` and then `BAfromZeroToDataAw` with some parameters and simulation choice of false

Note that when `simulation==False`, that is the only time that the list is created. otherwise the list is left empty.

Note also that `simulation_choice` defaults to `True` in forward simulation, i.e. for when `BAfromZeroToData__` are called from forward simulation.

``` python
def simulate_forwards_df(plot_df, simulation_choice='yes'):
    # ...
    for row in plot_df:
    # ...
    
            species_factors = get_factors_for_all_species(
            startTage=startTage,
            startTageAw=startTageAw,
            # ...
            )
    # ...
        BA_0_to_data_Aw = BAfromZeroToDataAw(
            startTage, SI_bh_Aw, N0_Aw, BA_Aw0, SDF_Aw0, f_Aw, densities, simulation_choice='no', simulation=False
            )
        BA_0_to_data_Sb = BAfromZeroToDataSb(
            startTage, startTageSb, y2bh_Sb, SC_Sb, SI_bh_Sb, N_bh_SbT, 
            N0_Sb, BA_Sb0, f_Sb, simulation_choice, simulation=False
            )
        BA_0_to_data_Sw = BAfromZeroToDataSw(
            startTage, startTageSw, y2bh_Sw, SC_Sw, SI_bh_Sw, N_bh_SwT, 
            N0_Sw, SDF_Aw0, SDF_Pl0, SDF_Sb0, BA_Sw0, f_Sw, simulation_choice, simulation=False
            )
        BA_0_to_data_Pl = BAfromZeroToDataPl(
            startTage, startTagePl, y2bh_Pl, SC_Pl, SI_bh_Pl, N_bh_PlT, 
            N0_Pl, SDF_Aw0, SDF_Sw0, SDF_Sb0, BA_Pl0, f_Pl, simulation_choice, simulation=False
            )
```

`get_factors_for_all_species` calls factor finder functions for each species, if the species is present, and returns a dict of the factors

`BAfactorFinder_Aw` is the main suspect, for sime reason aspen has a harder time converging, so the loop in this function runs many times


``` python
def BAfactorFinder_Aw(**kwargs):
    startTage = kwargs['startTage']
    # ...
    simulation_choice = 'yes'
    f_Aw = 100
    f_AwP1 = 100 * f_Aw
    acceptableDiff = 0.01 * BA_AwT
    BADiffFlag = False
    iterCount = 0

    while BADiffFlag == False:
        BA_AwB = BAfromZeroToDataAw(startTage, SI_bh_Aw, N0_Aw,
                                    BA_Aw0, SDF_Aw0, f_Aw, densities,
                                    simulation_choice, simulation=True)[0]

        if abs(BA_AwT - BA_AwB) < acceptableDiff:
            BADiffFlag = True
        else:
            if (BA_AwT - BA_AwB) < 0:
                f_AwP1 = f_Aw
                f_AwP = f_Aw  *  (1+(numpy.log10(BA_AwT) - numpy.log10(abs(BA_AwB)))/ (100*numpy.log10(abs(BA_AwB))))
                f_Aw = (f_AwP+f_Aw)/2
            elif (BA_AwT - BA_AwB) > 0:
                f_AwN = f_Aw * (1+(numpy.log10(BA_AwT) + numpy.log10(abs(BA_AwB)))/ (100* numpy.log10(abs(BA_AwB))))
                f_Aw = (f_Aw+f_AwP1)/2

        iterCount = iterCount + 1

        if iterCount == 10000:
            logger.warning(('GYPSYNonSpatial.BAfactorFinder_Aw()'
                 ' Slow convergence with Basal Area: %s'
                 ' and factor:%s '), BA_AwB, f_Aw)
            return f_Aw
    return f_Aw

```

It calls `BAfromZeroToDataAw` with `simulation_choice` of `'yes'` and `simulation=True` **BUT IT ONLY USES THE 1ST RETURN VALUE**

factor finder

``` python
def BAfromZeroToDataAw(startTage, SI_bh_Aw, N0_Aw, BA_Aw0, SDF_Aw0, f_Aw,
                       densities, simulation_choice, simulation=True):
    logger.debug('getting basal area from time zero to time of data for aspen')

    if simulation_choice == 'yes':
        max_age = startTage
    elif simulation_choice == 'no':
        max_age = 250

    basal_area_aw_df = pd.DataFrame(columns=['BA_Aw'], index=xrange(max_age))
    BA_tempAw = BA_Aw0

    for i, SC_Dict in enumerate(densities[0: max_age]):
        bhage_Aw = SC_Dict['bhage_Aw']
        SC_Aw = SC_Dict['SC_Aw']
        N_bh_AwT = SC_Dict['N_bh_AwT']

        if N0_Aw > 0:
            if bhage_Aw < 0:
                BA_AwB = 0
            if bhage_Aw > 0:
                SC_Aw = (SC_Aw) * f_Aw
                BAinc_Aw = BasalAreaIncrementNonSpatialAw('Aw', SC_Aw, SI_bh_Aw, N_bh_AwT,
                                                          N0_Aw, bhage_Aw, BA_tempAw)
                BA_tempAw = BA_tempAw + BAinc_Aw
                BA_AwB = BA_tempAw
                if BA_AwB < 0:
                    BA_AwB = 0
            else:
                BA_AwB = 0
        else:
            BA_tempAw = 0
            BA_AwB = 0

        # we might be able to remove this, always update this data frame, then we don't have to call
        # bafromzerotodata again in forward_simulation using the newly found factor
        if simulation == False:
            basal_area_aw_df.loc[i, 'BA_Aw'] = BA_AwB

    return BA_AwB, basal_area_aw_df
```

# Revise the code

Go on. Do it.

# Review code changes

In [2]:
%%bash
# git log --since 2016-11-09 --oneline

In [3]:
# ! git diff HEAD~7 ../gypsy

# Tests

Do tests still pass?

# Run profiling

In [18]:
from gypsy.forward_simulation import simulate_forwards_df

/opt/gypsy/venv/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [19]:
data = pd.read_csv('../private-data/prepped_random_sample_300.csv', index_col=0, nrows=10)

In [20]:
%%prun -D forward-sim-2.prof -T forward-sim-2.txt -q
result = simulate_forwards_df(data)

 
*** Profile stats marshalled to file u'forward-sim-1.prof'. 

*** Profile printout saved to text file u'forward-sim-1.txt'. 


In [21]:
!head forward-sim-2.txt

         10055657 function calls (9875729 primitive calls) in 76.264 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
   492069    6.857    0.000    6.857    0.000 GYPSYNonSpatial.py:427(BasalAreaIncrementNonSpatialAw)
  1836602    6.527    0.000    9.190    0.000 {isinstance}
796652/624746    3.102    0.000    4.823    0.000 {len}
     7191    2.670    0.000   40.459    0.006 GYPSYNonSpatial.py:959(BAfromZeroToDataAw)
   511948    2.020    0.000    3.373    0.000 {getattr}


In [22]:
!diff -y forward-sim-2.txt forward-sim-1.txt

diff: forward-sim.txt: No such file or directory


# Compare performance visualizations

Now use either of these commands to visualize the profiling

```
pyprof2calltree -k -i forward-sim-1.prof forward-sim-1.txt

# or

dc run --service-ports snakeviz notebooks/forward-sim-1.prof
```

### Old

![definitive reference profile screenshot](forward-sim-1-performance.png)

### New

![1st iteration performance](forward-sim-2-performance.png)

## Summary of performance improvements

forward_simulation is now 4x faster due to the changes outlined in the code review section above

on my hardware, this takes 1000 plots to ~8 minutes

on carol's hardware, this takes 1000 plots to ~25 minutes

For 1 million plots, we're looking at 5 to 17 days on desktop hardware



# Profile with I/O


In [ ]:
! rm -rfd gypsy-output

In [ ]:
output_dir = 'gypsy-output'

In [20]:
%%prun -D forward-sim-2.prof -T forward-sim-2.txt -q
# restart the kernel first
data = pd.read_csv('../private-data/prepped_random_sample_300.csv', index_col=0, nrows=10)
result = simulate_forwards_df(data)
os.makedirs(output_dir)
for plot_id, df in result.items():
    filename = '%s.csv' % plot_id
    output_path = os.path.join(output_dir, filename)
    df.to_csv(output_path)


 
*** Profile stats marshalled to file u'forward-sim-1.prof'. 

*** Profile printout saved to text file u'forward-sim-1.txt'. 


# Identify new areas to optimize



- from last time:
    - parallel (3 cores) gets us to 2 - 6 days - save for last
    - AWS with 36 cores gets us to 4 - 12 hours ($6.70 - $20.10 USD on a c4.8xlarge instance in US West Region)
- now:
    - 

# Identify some means of optimization

In order of priority/time taken

1.
2.